In [63]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd

#stock index
# country='Vietnam'
# country='Australia'
# country='Japan'

#google traffic
gtraffic=pd.read_csv(r'OneTimeSource\changes-visitors-covid.csv')
# c1=gtraffic['Entity'].str.contains(country)
gtraffic['date']=pd.to_datetime(gtraffic['Day'])
# gtraffic=gtraffic[c1]
gtraffic=gtraffic[['Entity', 'date', 'transit_stations', 'workplaces','retail_and_recreation']]
gtraffic=gtraffic.rename(columns={'Entity':'location'})
gtraffic.tail(2)

owid_o=pd.read_csv(r'OneTimeSource/owid-covid-data.csv')
# c1=owid_o['location'].str.contains(country)
owid=owid_o.reset_index(drop=True)[['location', 'date','new_cases','new_deaths','stringency_index']]
owid['date']=pd.to_datetime(owid['date'])

df=owid.merge(gtraffic,on=['date','location'],how='left')
df=df.sort_values(by='date').reset_index(drop=True)

# df['stockindex_7mean'] = df['stockindex'].rolling(7, 1).mean()
df['new_cases_7mean'] = df.groupby('location')['new_cases'].transform(lambda x: x.rolling(7, 7).mean())
df['new_deaths_7mean'] = df.groupby('location')['new_deaths'].transform(lambda x: x.rolling(7, 7).mean())
df['transit_stations_7mean'] = df.groupby('location')['transit_stations'].transform(lambda x: x.rolling(7, 7).mean())
df['workplaces_7mean'] = df.groupby('location')['workplaces'].transform(lambda x: x.rolling(7, 7).mean())
df['retail_and_recreation_7mean'] = df.groupby('location')['retail_and_recreation'].transform(lambda x: x.rolling(7, 7).mean())

# df
# df=df[['date','location','stringency_index', 'new_cases_7mean', 'new_deaths_7mean','new_cases','new_deaths',
#          'transit_stations_7mean','workplaces_7mean', 'retail_and_recreation_7mean','transit_stations','workplaces','retail_and_recreation',
#        ]]
# df=df[df['location'].str.contains('Japan')]
df=df[df['date']>'2021-12-31']
df=df[['date','location','new_cases_7mean','retail_and_recreation_7mean']]
# df.to_clipboard(index=False)
# df

In [26]:
# peakdate=pd.read_clipboard()
peakdate

,国家,第一波,第二波
0,Japan,"94,552","226,955"
1,Singapore,"18,320","10,756"
2,South Korea,"404,666","120,089"
3,Hong Kong,"66,194","10,169"
4,Australia,"107,666","57,591"


In [36]:
df.tail(2)

,date,location,new_cases_7mean,retail_and_recreation_7mean
245782,2022-12-27,Norway,205.285714,NaN
245783,2022-12-27,Zimbabwe,0.000000,NaN


In [64]:
def finddate(x,y):
    country=x['国家']
    tofind_value=x[y].replace(',','')
    tofind_value=int(tofind_value)
    # print(country,tofind_value)
    d=df[df['location']==country]
    d['temp']=abs(d['new_cases_7mean']/tofind_value-1)
    return d.loc[d['temp'].idxmin()]['date'].date()
peakdate['第一波_date']=peakdate.apply(lambda x:finddate(x,'第一波'),axis=1)
peakdate['第二波_date']=peakdate.apply(lambda x:finddate(x,'第二波'),axis=1)
peakdate

,国家,第一波,第二波,第一波_date,第二波_date,第一波_GM,第二波_GM
0,Japan,"94,552","226,955",2022-02-09,2022-08-24,None,89.285714
1,Singapore,"18,320","10,756",2022-02-27,2022-07-19,None,2.571429
2,South Korea,"404,666","120,089",2022-03-18,2022-08-24,None,34.714286
3,Hong Kong,"66,194","10,169",2022-03-04,2022-09-07,None,80.571429
4,Australia,"107,666","57,591",2022-01-15,2022-04-06,None,68.857143


In [66]:
def findmobility(x,y):
    country=x['国家']
    tofind_value=x[y].replace(',','')
    tofind_value=int(tofind_value)
    # print(country,tofind_value)
    d=df[df['location']==country]
    d['temp']=abs(d['new_cases_7mean']/tofind_value-1)
    # print(d.loc[d['temp'].idxmin()])
    return d.loc[d['temp'].idxmin()]['retail_and_recreation_7mean']
peakdate['第一波_GM']=peakdate.apply(lambda x:findmobility(x,'第一波'),axis=1)
peakdate['第二波_GM']=peakdate.apply(lambda x:findmobility(x,'第二波'),axis=1)
peakdate

,国家,第一波,第二波,第一波_date,第二波_date,第一波_GM,第二波_GM
0,Japan,"94,552","226,955",2022-02-09,2022-08-24,-17.530571,-6.224429
1,Singapore,"18,320","10,756",2022-02-27,2022-07-19,-16.265429,-9.469429
2,South Korea,"404,666","120,089",2022-03-18,2022-08-24,-12.979571,5.122571
3,Hong Kong,"66,194","10,169",2022-03-04,2022-09-07,-36.694000,-11.490000
4,Australia,"107,666","57,591",2022-01-15,2022-04-06,-13.796000,-9.224286


In [57]:
df

,date,location,new_cases_7mean,retail_and_recreation_7mean
158018,2022-01-01,Ecuador,1362.571429,7.142857
158019,2022-01-01,New Zealand,57.142857,30.428571
158020,2022-01-01,Ethiopia,4084.285714,NaN
158021,2022-01-01,Cyprus,2758.714286,NaN
158022,2022-01-01,Trinidad and Tobago,405.285714,57.857143
...,...,...,...,...
245779,2022-12-27,Nigeria,0.000000,NaN
245780,2022-12-27,Bonaire Sint Eustatius and Saba,7.857143,NaN
245781,2022-12-27,Niger,0.000000,NaN
245782,2022-12-27,Norway,205.285714,NaN


In [67]:
peakdate.to_clipboard()